In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
tf.keras.backend.set_floatx('float64')
from misc import *
from collections import deque
from nets import Q2, Actor
import random

In [10]:

class Q2(tf.keras.Model):
    def __init__(self, valreg=0.01):
        super(Q2,self).__init__()

        self.l1 = Dense(10, input_shape=(2,),kernel_initializer=tf.random_uniform_initializer(),
        bias_initializer = tf.random_uniform_initializer(),
        kernel_regularizer=tf.keras.regularizers.l1(valreg),
    activity_regularizer=tf.keras.regularizers.l2(valreg))

        self.l2 = Dense(10, kernel_regularizer=tf.keras.regularizers.l1(valreg),
    activity_regularizer=tf.keras.regularizers.l2(valreg),
    kernel_initializer=tf.random_uniform_initializer(),
    bias_initializer = tf.random_uniform_initializer())
        self.l3 = Dense(10, kernel_regularizer=tf.keras.regularizers.l1(valreg),
    activity_regularizer=tf.keras.regularizers.l2(valreg),
    kernel_initializer=tf.random_uniform_initializer(),
    bias_initializer = tf.random_uniform_initializer())
        
        self.l4 = Dense(12, kernel_regularizer=tf.keras.regularizers.l1(valreg),
    activity_regularizer=tf.keras.regularizers.l2(valreg),
    kernel_initializer=tf.random_uniform_initializer(),
    bias_initializer = tf.random_uniform_initializer())
        
        self.l5 = Dense(2, kernel_regularizer=tf.keras.regularizers.l1(valreg),
    activity_regularizer=tf.keras.regularizers.l2(valreg),
    kernel_initializer=tf.random_uniform_initializer(),
    bias_initializer = tf.random_uniform_initializer())

    def call(self, input):
        feat = tf.nn.relu(self.l1(input))
        #feat = tf.nn.dropout(feat, rate=0.01)
        feat = tf.nn.relu(self.l2(feat))
        feat = tf.nn.relu(self.l3(feat))
        feat = tf.nn.relu(self.l4(feat))
        feat = self.l5(feat)
        return feat
        
    def predict(self, history_one):
        inp = np.expand_dims(history_one, axis=0)
        return np.squeeze(self(inp).numpy())

    def __str__(self):
        return self.name


In [11]:
def createdataset(size_dataset=10**3, splits=1):
    dataset = []
    labels=[]
    for i in range(size_dataset):
        ph = np.random.choice([-1.,1.],1)[0]
        b = -.7

        p0 = Prob(ph*0.4, b, 0)
        outcome = np.random.choice([0.,1.], 1, p=[p0,1-p0])[0]
        labels.append(int(ph+1)/2)
        dataset.append([outcome])
    dataset = np.split(np.array(dataset), splits+(len(dataset)%splits ))
    labels = np.split(np.array(labels), splits+(len(labels)%splits ))
    for i,k in enumerate(labels):
        labels[i] = tf.convert_to_tensor(k)
    return dataset, labels

In [57]:
data, label = createdataset(10**3,10)

lossfunction = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(lr=10**-3)
q2 = Q2()

def givemeloss(model, x, y, training=False):
    y_ = model(x, training=training)
    return lossfunction(y_true = y, y_pred = y_)
train_loss_results = []
train_accuracy_results = []

for epoch in range(50):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    for k in range(len(data)):
        with tf.GradientTape() as tape:
            tape.watch(q2.trainable_variables)
            loss = givemeloss(q2, data[k], label[k])
            grads = tape.gradient(loss, q2.trainable_variables)
        optimizer.apply_gradients(zip(grads, q2.trainable_variables))
        epoch_loss_avg(loss)
        epoch_accuracy(label[k], q2(data[k]))
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    if epoch % 5 == 0:
        print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))


Epoch 000: Loss: 0.693, Accuracy: 49.800%
Epoch 005: Loss: 0.693, Accuracy: 49.800%
Epoch 010: Loss: 0.693, Accuracy: 49.800%
Epoch 015: Loss: 0.683, Accuracy: 81.000%
Epoch 020: Loss: 0.561, Accuracy: 81.000%
Epoch 025: Loss: 0.475, Accuracy: 81.000%
Epoch 030: Loss: 0.475, Accuracy: 81.000%
Epoch 035: Loss: 0.475, Accuracy: 81.000%
Epoch 040: Loss: 0.475, Accuracy: 81.000%
Epoch 045: Loss: 0.475, Accuracy: 81.000%


Epoch 000: Loss: 0.684, Accuracy: 59.400%
Epoch 005: Loss: 0.500, Accuracy: 79.300%


In [58]:
for ot in [0., 1.]:
    print(tf.nn.softmax(q2(np.array([[ot]]))))

tf.Tensor([[0.76126885 0.23873115]], shape=(1, 2), dtype=float64)
tf.Tensor([[0.11962863 0.88037137]], shape=(1, 2), dtype=float64)


In [40]:
np.where(np.argmax(q2(data[0]),axis=1)-label[0].numpy() == -1.)

(array([ 8, 24, 34, 46, 47, 48]),)

In [69]:
diff =np.argmax(tf.nn.softmax(q2.predict(data[0])),axis=1) - label[0].numpy() 

In [74]:
print(np.where(diff == 1.))
print(np.where(diff == -1.))

(array([ 6, 36]),)
(array([ 5, 23, 26, 29, 31, 34, 45, 46, 50, 53, 78, 83, 86, 87]),)


In [71]:
print(data[data[0][6]

array([1.])

In [72]:
data[0][36]

array([1.])

In [76]:
data[0][np.where(diff == -1.)]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [81]:
label[0].numpy()[np.where(diff == -1.)]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [85]:
np.argmax(tf.nn.softmax(q2(np.array([[0.]]))))

0

In [62]:
b=-.7
outcome=0
po1=np.sum([Prob(pp*0.4,b,outcome) for pp in [-1,1]])
print(np.array([Prob(k*0.4, b, outcome) for k in [-1,1]])/po1)

[0.75398872 0.24601128]


In [63]:
b=-.7
outcome=1
po1=np.sum([Prob(pp*0.4,b,outcome) for pp in [-1,1]])
print(np.array([Prob(k*0.4, b, outcome) for k in [-1,1]])/po1)

[0.10924219 0.89075781]


In [86]:
#Okay it's fine that accuracy is not the 100 porque i could send -1 and obtain outcome 1 with some probability!!!

In [90]:
#actually, with this probability:
(Prob(-0.4,b,1) + Prob(0.4,b,0)   )/2

0.19213304707932957

In [ ]:
Ahi lo tenés papá